In [1]:
from __future__ import print_function
import sys, os
import numpy as np
from array import array
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

import ROOT
from ROOT import TCanvas, TH1F, TH2F, TF1, TMath, TGraph, TFile, TSpectrum, TPaveText, TMultiGraph, TGraphErrors, TLine
from ROOT import gStyle, gROOT, gDirectory, gPad

import constants
import crv_event
import crv_spill
import utils
import root_utils
import geometry
import geometry_constants
import filepath

gROOT.Reset()
gROOT.SetBatch(1)
gROOT.ProcessLine( "gErrorIgnoreLevel = 1001;")
gStyle.SetOptStat(111111110)
gStyle.SetOptFit(0)
gStyle.SetLineScalePS(0.3)

analysis_dir = "~/CRVteststand/analysis/testbench/"
analysisFileName = analysis_dir + "temperatureScanAnalysis.root"

analysisFile = TFile(analysisFileName, "READ")
spilltree = analysisFile.Get("spill_summary")
subruntree = analysisFile.Get('subrun_summary')
nSpill = spilltree.GetEntries()
nSubrun = subruntree.GetEntries()
nCh = geometry_constants.nChannelPerFEB

calibConstantRaw = {"cosmics":[[[] for j in range(nCh)] for i in range(2)],
                    "LED":[[[] for j in range(nCh)] for i in range(2)]}
calibConstantCorrected = {"cosmics":[[[] for j in range(nCh)] for i in range(2)],
                          "LED":[[[] for j in range(nCh)] for i in range(2)]}
CMBtemp = {"cosmics":[[[] for j in range(nCh)] for i in range(2)], 
           "LED":[[[] for j in range(nCh)] for i in range(2)]}
FEBtemp = {"cosmics":[[[] for j in range(nCh)] for i in range(2)], 
           "LED":[[[] for j in range(nCh)] for i in range(2)]}
busSiPMBias = {"cosmics":[[[] for j in range(nCh)] for i in range(2)], 
               "LED":[[[] for j in range(nCh)] for i in range(2)]}
pedestal = {"cosmics":[[[] for j in range(nCh)] for i in range(2)], 
            "LED":[[[] for j in range(nCh)] for i in range(2)]}

calibCMB_residue = {"cosmics":[[[] for j in range(nCh)] for i in range(2)], 
                    "LED":[[[] for j in range(nCh)] for i in range(2)]}

for iSubrun in range(nSubrun):
    subruntree.GetEntry(iSubrun)
    tType = subruntree.runType[0]
    
    for iFEB in range(2):
        for iCh in range(nCh):
            calibConstantRaw[tType][iFEB][iCh].append(subruntree.calibRaw[iFEB*nCh+iCh])
            calibConstantCorrected[tType][iFEB][iCh].append(subruntree.calibAdj[iFEB*nCh+iCh])
            CMBtemp[tType][iFEB][iCh].append(subruntree.temperatureCMB[iFEB*nCh+iCh])
            FEBtemp[tType][iFEB][iCh].append(subruntree.temperatureFEB[iFEB])
            busSiPMBias[tType][iFEB][iCh].append(subruntree.busSiPMBias[iFEB*8+int(iCh/8)])
            pedestal[tType][iFEB][iCh].append(subruntree.pedestal[iFEB*nCh+iCh])

# put things into scatter plots, then fit LED graphs
xdiv = 3
ydiv = 2

calibmin = min(np.amin(calibConstantRaw['cosmics']), np.amin(calibConstantRaw['LED'])-10.)
calibmax = max(np.amax(calibConstantRaw['cosmics']), np.amax(calibConstantRaw['LED'])+10.)

pdfname = analysis_dir+"temperatureScanAnalysis_calibTemp.pdf"
fC00 = TCanvas("c00", "c00", 1200*xdiv, 900*ydiv)
fC00.Divide(xdiv,ydiv,0.0001,0.001)
fC00.Print(pdfname+"[", "pdf")

fMG_calib_CMBtemp = [[None]*nCh for i in range(2)]
fGraph_calib_CMBtemp_cosmic = [[None]*nCh for i in range(2)]
fGraph_calib_CMBtemp_LED = [[None]*nCh for i in range(2)]
linearFit = [[None]*nCh for i in range(2)]
fMG_calibAdj_CMBtemp = [[None]*nCh for i in range(2)]
fGraph_calibAdj_CMBtemp_cosmic = [[None]*nCh for i in range(2)]
fGraph_calibAdj_CMBtemp_LED = [[None]*nCh for i in range(2)]
fGraph_res_CMBtemp_LED = [[None]*nCh for i in range(2)]
fGraph_res_FEBtemp_LED = [[None]*nCh for i in range(2)]
fGraph_res_busSiPMBias_LED = [[None]*nCh for i in range(2)]
fGraph_pedestal_CMBtemp_LED = [[None]*nCh for i in range(2)]

fitting_par_k = []
fitting_par_k_err = []
fitting_par_b = []
fitting_par_b_err = []
print("FEB   Ch %10s %10s %10s %10s"%("k", "k_err", "b", "b_err"))

for iFEB in range(2):
    for iCh in range(nCh):
        fCanvas= TCanvas("c_%i_%i"%(iFEB, iCh), "c_%i_%i"%(iFEB, iCh), 1200*xdiv, 900*ydiv)
        fCanvas.Divide(xdiv,ydiv,0.0001,0.001)
        gStyle.SetOptStat(0)
        gStyle.SetOptFit(111)
        
        fCanvas.cd(1)
        fMG_calib_CMBtemp[iFEB][iCh] = TMultiGraph("fMG_calib_CMBtemp_%i_%i"%(iFEB, iCh), "(CMB) Temperature Dependence of Calibration Constant; CMB Temperature [#circC]; Calibration Constant [ADC#timesns/PE]")
        
        fGraph_calib_CMBtemp_LED[iFEB][iCh] = TGraph(len(CMBtemp['LED'][iFEB][iCh]), array('d', CMBtemp['LED'][iFEB][iCh]), array('d', calibConstantRaw['LED'][iFEB][iCh]))
        fGraph_calib_CMBtemp_LED[iFEB][iCh].SetName("scat_calib_CMBtemp_%i_%i_LED"%(iFEB, iCh))
        fGraph_calib_CMBtemp_LED[iFEB][iCh].SetTitle("(CMB) Temperature Dependence of Calibration Constant; CMB Temperature [#circC]; Calibration Constant [ADC#timesns/PE]")
        fGraph_calib_CMBtemp_LED[iFEB][iCh].SetMarkerStyle(21)
        fGraph_calib_CMBtemp_LED[iFEB][iCh].SetMarkerColor(2)
        fGraph_calib_CMBtemp_LED[iFEB][iCh].SetLineColor(2)
        fGraph_calib_CMBtemp_LED[iFEB][iCh].SetLineWidth(3)
        fGraph_calib_CMBtemp_LED[iFEB][iCh].SetMarkerSize(1)
        fMG_calib_CMBtemp[iFEB][iCh].Add(fGraph_calib_CMBtemp_LED[iFEB][iCh])
        
        fGraph_calib_CMBtemp_cosmic[iFEB][iCh] = TGraph(len(CMBtemp['cosmics'][iFEB][iCh]), array('d', CMBtemp['cosmics'][iFEB][iCh]), array('d', calibConstantRaw['cosmics'][iFEB][iCh]))
        fGraph_calib_CMBtemp_cosmic[iFEB][iCh].SetName("scat_calib_CMBtemp_%i_%i_cosmics"%(iFEB, iCh))
        fGraph_calib_CMBtemp_cosmic[iFEB][iCh].SetTitle("(CMB) Temperature Dependence of Calibration Constant; CMB Temperature [#circC]; Calibration Constant [ADC#timesns/PE]")
        fGraph_calib_CMBtemp_cosmic[iFEB][iCh].SetMarkerStyle(20)
        fGraph_calib_CMBtemp_cosmic[iFEB][iCh].SetMarkerColor(4)
        fGraph_calib_CMBtemp_cosmic[iFEB][iCh].SetLineColor(4)
        fGraph_calib_CMBtemp_cosmic[iFEB][iCh].SetLineWidth(3)
        fGraph_calib_CMBtemp_cosmic[iFEB][iCh].SetMarkerSize(1)
        fMG_calib_CMBtemp[iFEB][iCh].Add(fGraph_calib_CMBtemp_cosmic[iFEB][iCh])
        
        fMG_calib_CMBtemp[iFEB][iCh].GetXaxis().SetLimits(13.,23.)
        # fMG_calib_CMBtemp[iFEB][iCh].GetHistogram().SetMaximum(calibmax)
        # fMG_calib_CMBtemp[iFEB][iCh].GetHistogram().SetMinimum(calibmin)
        fMG_calib_CMBtemp[iFEB][iCh].Draw("AP")
                                                               
        fLegend1 = ROOT.TLegend(0.75, 0.6, 0.98, 0.72)
        fLegend1.SetHeader("FEB%i_Ch%02i"%(iFEB, iCh),"C")
        fLegend1.AddEntry(fGraph_calib_CMBtemp_LED[iFEB][iCh], "LED runs")
        fLegend1.AddEntry(fGraph_calib_CMBtemp_cosmic[iFEB][iCh], "Cosmic runs")
        fLegend1.Draw("SAME")
        
        #fit LED
        linearFit[iFEB][iCh] = TF1("linearFit%i_Ch%02i"%(iFEB, iCh), "[0]*x+[1]", 13., 23.)
        linearFit[iFEB][iCh].SetParameter(0, -10.)
        linearFit[iFEB][iCh].SetParameter(1, 600.)
        linearFit[iFEB][iCh].SetParName(0, "k")
        linearFit[iFEB][iCh].SetParName(1, "b")
        linearFit[iFEB][iCh].SetParLimits(0, -100.,0.)
        linearFit[iFEB][iCh].SetParLimits(1, 0.,1000.)
        tfrp = fGraph_calib_CMBtemp_LED[iFEB][iCh].Fit(linearFit[iFEB][iCh], "QRSF")
        linearFit[iFEB][iCh].SetLineColor(1)
        linearFit[iFEB][iCh].SetLineWidth(1)
        linearFit[iFEB][iCh].Draw("SAME")
        # fCanvas.Update()
        tk = linearFit[iFEB][iCh].GetParameter(0)
        tk_err = linearFit[iFEB][iCh].GetParError(0)
        tb = linearFit[iFEB][iCh].GetParameter(1)
        tb_err = linearFit[iFEB][iCh].GetParError(1)
        fitting_par_k.append(tk)
        fitting_par_k_err.append(tk_err)
        fitting_par_b.append(tb)
        fitting_par_b_err.append(tb_err)
        print("%3i %4i %10.3f %10.3f %10.3f %10.3f"%(iFEB, iCh, tk, tk_err, tb, tb_err))
        
        fCanvas.cd(2)
        fMG_calibAdj_CMBtemp[iFEB][iCh] = TMultiGraph("fMG_calibAdj_CMBtemp_%i_%i"%(iFEB, iCh), "Original Temperature Correction Check; CMB Temperature [#circC]; Adjusted Calibration Constant [ADC#timesns/PE]")
        
        fGraph_calibAdj_CMBtemp_LED[iFEB][iCh] = TGraph(len(CMBtemp['LED'][iFEB][iCh]), array('d', CMBtemp['LED'][iFEB][iCh]), array('d', calibConstantCorrected['LED'][iFEB][iCh]))
        fGraph_calibAdj_CMBtemp_LED[iFEB][iCh].SetName("scat_calibAdj_CMBtemp_%i_%i_LED"%(iFEB, iCh))
        fGraph_calibAdj_CMBtemp_LED[iFEB][iCh].SetTitle("Original Temperature Correction Check; CMB Temperature [#circC]; Adjusted Calibration Constant [ADC#timesns/PE]")
        fGraph_calibAdj_CMBtemp_LED[iFEB][iCh].SetMarkerStyle(21)
        fGraph_calibAdj_CMBtemp_LED[iFEB][iCh].SetMarkerColor(2)
        fGraph_calibAdj_CMBtemp_LED[iFEB][iCh].SetLineColor(2)
        fGraph_calibAdj_CMBtemp_LED[iFEB][iCh].SetLineWidth(3)
        fGraph_calibAdj_CMBtemp_LED[iFEB][iCh].SetMarkerSize(1)
        fMG_calibAdj_CMBtemp[iFEB][iCh].Add(fGraph_calibAdj_CMBtemp_LED[iFEB][iCh])
        
        fGraph_calibAdj_CMBtemp_cosmic[iFEB][iCh] = TGraph(len(CMBtemp['cosmics'][iFEB][iCh]), array('d', CMBtemp['cosmics'][iFEB][iCh]), array('d', calibConstantCorrected['cosmics'][iFEB][iCh]))
        fGraph_calibAdj_CMBtemp_cosmic[iFEB][iCh].SetName("scat_calibAdj_CMBtemp_%i_%i_cosmics"%(iFEB, iCh))
        fGraph_calibAdj_CMBtemp_cosmic[iFEB][iCh].SetTitle("Original Temperature Correction Check; CMB Temperature [#circC]; Adjusted Calibration Constant [ADC#timesns/PE]")
        fGraph_calibAdj_CMBtemp_cosmic[iFEB][iCh].SetMarkerStyle(20)
        fGraph_calibAdj_CMBtemp_cosmic[iFEB][iCh].SetMarkerColor(4)
        fGraph_calibAdj_CMBtemp_cosmic[iFEB][iCh].SetLineColor(4)
        fGraph_calibAdj_CMBtemp_cosmic[iFEB][iCh].SetLineWidth(3)
        fGraph_calibAdj_CMBtemp_cosmic[iFEB][iCh].SetMarkerSize(2)
        fMG_calibAdj_CMBtemp[iFEB][iCh].Add(fGraph_calibAdj_CMBtemp_cosmic[iFEB][iCh])
        
        fMG_calibAdj_CMBtemp[iFEB][iCh].GetXaxis().SetLimits(13.,23.)
        fMG_calibAdj_CMBtemp[iFEB][iCh].Draw("AP")
                                                               
        fLegend2 = ROOT.TLegend(0.75, 0.6, 0.98, 0.72)
        fLegend2.SetHeader("FEB%i_Ch%02i"%(iFEB, iCh),"C")
        fLegend2.AddEntry(fGraph_calib_CMBtemp_LED[iFEB][iCh], "LED runs")
        fLegend2.AddEntry(fGraph_calib_CMBtemp_cosmic[iFEB][iCh], "Cosmic runs")
        fLegend2.Draw("SAME")
                
        #do residue, try find correspondence
        for i in range(len(CMBtemp['LED'][iFEB][iCh])):
            calibCMB_residue['LED'][iFEB][iCh].append(calibConstantRaw['LED'][iFEB][iCh][i]-linearFit[iFEB][iCh].Eval(CMBtemp['LED'][iFEB][iCh][i]))
        
        fCanvas.cd(3)
        fGraph_res_CMBtemp_LED[iFEB][iCh] = TGraph(len(CMBtemp['LED'][iFEB][iCh]), array('d', CMBtemp['LED'][iFEB][iCh]), array('d', calibCMB_residue['LED'][iFEB][iCh]))
        fGraph_res_CMBtemp_LED[iFEB][iCh].SetName("scat_res_CMBtemp_%i_%i_LED"%(iFEB, iCh))
        fGraph_res_CMBtemp_LED[iFEB][iCh].SetTitle("Raw Calib. Constant - CMB Temperature Residue; CMB Temperature [#circC]; Fit Residue [ADC#timesns/PE]")
        fGraph_res_CMBtemp_LED[iFEB][iCh].SetMarkerStyle(21)
        fGraph_res_CMBtemp_LED[iFEB][iCh].SetMarkerColor(2)
        fGraph_res_CMBtemp_LED[iFEB][iCh].SetLineColor(2)
        fGraph_res_CMBtemp_LED[iFEB][iCh].SetLineWidth(3)
        fGraph_res_CMBtemp_LED[iFEB][iCh].SetMarkerSize(1)
        fGraph_res_CMBtemp_LED[iFEB][iCh].Draw("AP")
        
        fCanvas.cd(4)
        fGraph_res_FEBtemp_LED[iFEB][iCh] = TGraph(len(FEBtemp['LED'][iFEB][iCh]), array('d', FEBtemp['LED'][iFEB][iCh]), array('d', calibCMB_residue['LED'][iFEB][iCh]))
        fGraph_res_FEBtemp_LED[iFEB][iCh].SetName("scat_res_FEBtemp_%i_%i_LED"%(iFEB, iCh))
        fGraph_res_FEBtemp_LED[iFEB][iCh].SetTitle("Residue w.r.t FEB Temperature; FEB Temperature [#circC]; Fit Residue [ADC#timesns/PE]")
        fGraph_res_FEBtemp_LED[iFEB][iCh].SetMarkerStyle(21)
        fGraph_res_FEBtemp_LED[iFEB][iCh].SetMarkerColor(2)
        fGraph_res_FEBtemp_LED[iFEB][iCh].SetLineColor(2)
        fGraph_res_FEBtemp_LED[iFEB][iCh].SetLineWidth(3)
        fGraph_res_FEBtemp_LED[iFEB][iCh].SetMarkerSize(1)
        fGraph_res_FEBtemp_LED[iFEB][iCh].GetXaxis().SetLimits(26.,44.)
        fGraph_res_FEBtemp_LED[iFEB][iCh].Draw("AP")
        
        fCanvas.cd(5)
        fGraph_res_busSiPMBias_LED[iFEB][iCh] = TGraph(len(busSiPMBias['LED'][iFEB][iCh]), array('d', busSiPMBias['LED'][iFEB][iCh]), array('d', calibCMB_residue['LED'][iFEB][iCh]))
        fGraph_res_busSiPMBias_LED[iFEB][iCh].SetName("scat_res_busSiPMBias_%i_%i_LED"%(iFEB, iCh))
        fGraph_res_busSiPMBias_LED[iFEB][iCh].SetTitle("Raw Calib. Constant - Bus SiPM Bias; Bus SiPM Bias [V]; Fit Residue [ADC#timesns/PE]")
        fGraph_res_busSiPMBias_LED[iFEB][iCh].SetMarkerStyle(21)
        fGraph_res_busSiPMBias_LED[iFEB][iCh].SetMarkerColor(2)
        fGraph_res_busSiPMBias_LED[iFEB][iCh].SetLineColor(2)
        fGraph_res_busSiPMBias_LED[iFEB][iCh].SetLineWidth(3)
        fGraph_res_busSiPMBias_LED[iFEB][iCh].SetMarkerSize(1)
        fGraph_res_busSiPMBias_LED[iFEB][iCh].Draw("AP")
        
        fCanvas.cd(6)
        fGraph_pedestal_CMBtemp_LED[iFEB][iCh] = TGraph(len(CMBtemp['LED'][iFEB][iCh]), array('d', CMBtemp['LED'][iFEB][iCh]), array('d', pedestal['LED'][iFEB][iCh]))
        fGraph_pedestal_CMBtemp_LED[iFEB][iCh].SetName("scat_ped_CMBtemp_%i_%i_LED"%(iFEB, iCh))
        fGraph_pedestal_CMBtemp_LED[iFEB][iCh].SetTitle("Pedestal at different CMB Temperature; CMB Temperature [#circC]; Pedestal [ADC]")
        fGraph_pedestal_CMBtemp_LED[iFEB][iCh].SetMarkerStyle(21)
        fGraph_pedestal_CMBtemp_LED[iFEB][iCh].SetMarkerColor(2)
        fGraph_pedestal_CMBtemp_LED[iFEB][iCh].SetLineColor(2)
        fGraph_pedestal_CMBtemp_LED[iFEB][iCh].SetLineWidth(3)
        fGraph_pedestal_CMBtemp_LED[iFEB][iCh].SetMarkerSize(1)
        fGraph_pedestal_CMBtemp_LED[iFEB][iCh].Draw("AP")
        
        fCanvas.Print(pdfname, "Title: FEB%i_Ch%02i"%(iFEB, iCh))
        
analysisFile.Close()

fC00.cd(1)
gStyle.SetOptStat(111110)
gStyle.SetOptFit(0)
hist_k = TH1F("hist_k", "Temperature Constant of Calibration Constant; C/T [(ADC#timesns/PE)/#circC]; count", 20, -10., -8.)
for value in fitting_par_k:
    hist_k.Fill(value)
hist_k.Draw()
fC00.Print(pdfname+")", "Title: C/T")
fC00.Print(pdfname+"]", "pdf")

from IPython.display import IFrame
IFrame("temperatureScanAnalysis_calibTemp.pdf", width=1200, height=800)
        

Welcome to JupyROOT 6.28/00
FEB   Ch          k      k_err          b      b_err
  0    0     -8.868      0.035    585.435      0.617
  0    1     -9.181      0.036    585.023      0.636
  0    2     -9.046      0.037    591.950      0.650
  0    3     -8.966      0.035    578.509      0.614
  0    4     -8.938      0.034    589.877      0.614
  0    5     -8.969      0.033    574.499      0.585
  0    6     -8.926      0.035    571.511      0.623
  0    7     -8.899      0.035    574.541      0.629
  0    8     -8.608      0.035    557.463      0.628
  0    9     -8.870      0.038    572.904      0.677
  0   10     -8.892      0.039    576.830      0.694
  0   11     -8.613      0.036    557.518      0.652
  0   12     -8.952      0.045    624.856      0.793
  0   13     -9.279      0.043    646.863      0.764
  0   14     -8.956      0.046    638.264      0.826
  0   15     -9.127      0.046    643.207      0.825
  0   16     -9.435      0.057    641.960      1.021
  0   17     -9.73